# Limpieza y Unión de Datos (Manejo de columnas duplicadas)

Este cuaderno realiza la limpieza y unión de datos de tres archivos diferentes: data.txt, data1.txt y data3.txt, con manejo robusto de errores, inconsistencias en los datos y eliminación de columnas duplicadas.

## Paso 1: Importar las bibliotecas necesarias

In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
import os
import csv

## Paso 2: Definir funciones auxiliares (actualizadas)

In [2]:
def crear_carpetas():
    for carpeta in ['input', 'output']:
        if not os.path.exists(carpeta):
            os.makedirs(carpeta)
            print(f"Carpeta '{carpeta}' creada.")

def leer_archivo_con_errores(filename):
    filepath = os.path.join('input', filename)
    with open(filepath, 'r') as file:
        lines = file.readlines()
    
    header = [col.strip().lower().replace(' ', '_') for col in lines[0].strip().split(',')]
    data = []
    for i, line in enumerate(lines[1:], start=2):
        fields = [field.strip() for field in line.strip().split(',')]
        if len(fields) != len(header):
            print(f"Advertencia: La línea {i} en {filename} tiene {len(fields)} campos en lugar de {len(header)}.")
            print(f"Contenido de la línea: {line.strip()}")
            fields = fields[:len(header)]  # Truncar si hay más campos de lo esperado
        data.append(fields)
    
    df = pd.DataFrame(data, columns=header)
    return df

def limpiar_dataframe(df):
    # Eliminar columnas duplicadas
    df = df.loc[:, ~df.columns.duplicated()]
    
    for col in df.select_dtypes(include=['object']):
        df[col] = df[col].str.strip().replace('', np.nan)
    
    for col in ['age', 'height', 'weight', 'salary']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    if 'city' in df.columns:
        df['city'] = df['city'].apply(lambda x: 'Houston' if isinstance(x, str) and fuzz.ratio(x, 'Houston') > 80 else x)
        df['city'] = df['city'].apply(lambda x: 'Miami' if isinstance(x, str) and fuzz.ratio(x, 'Miami') > 80 else x)
    
    return df

def unificar_unidades(df):
    if 'height' in df.columns and df['height'].median() < 100:
        df['height'] = df['height'] * 2.54
    
    if 'weight' in df.columns and df['weight'].median() > 100:
        df['weight'] = df['weight'] * 0.453592
    
    return df

def eliminar_outliers(df):
    for col in ['age', 'height', 'weight', 'salary']:
        if col in df.columns:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df

def eliminar_columnas_duplicadas(df):
    # Crear un diccionario para almacenar columnas únicas
    columnas_unicas = {}
    for columna in df.columns:
        nombre_limpio = columna.strip().lower().replace(' ', '_').replace('"', '')
        if nombre_limpio not in columnas_unicas:
            columnas_unicas[nombre_limpio] = columna
    
    # Crear un nuevo DataFrame con las columnas únicas
    df_limpio = df[list(columnas_unicas.values())].copy()
    
    # Renombrar las columnas al formato limpio
    df_limpio.columns = list(columnas_unicas.keys())
    
    return df_limpio

## Paso 3: Crear carpetas de entrada y salida

In [3]:
crear_carpetas()

## Paso 4: Leer y limpiar los archivos

In [4]:
df1 = leer_archivo_con_errores('data1.txt')
df2 = leer_archivo_con_errores('data3.txt')
df3 = leer_archivo_con_errores('data.txt')

df1 = limpiar_dataframe(df1)
df2 = limpiar_dataframe(df2)
df3 = limpiar_dataframe(df3)

print("Columnas en df1:", df1.columns.tolist())
print("Columnas en df2:", df2.columns.tolist())
print("Columnas en df3:", df3.columns.tolist())

print("\nPrimeras filas de df1:")
print(df1.head())
print("\nPrimeras filas de df2:")
print(df2.head())
print("\nPrimeras filas de df3:")
print(df3.head())

Advertencia: La línea 7 en data1.txt tiene 7 campos en lugar de 6.
Contenido de la línea: "Frank", 35, 69.0, 154.3, 58000, "Washington, D.C."
Advertencia: La línea 7 en data.txt tiene 7 campos en lugar de 6.
Contenido de la línea: "Frank", 35, 175.5, 70.0, , "Washington, D.C."
Advertencia: La línea 13 en data.txt tiene 8 campos en lugar de 6.
Contenido de la línea: "Liam", 24, 178.5, 73.5, 61000, , ,
Advertencia: La línea 18 en data.txt tiene 8 campos en lugar de 6.
Contenido de la línea: "Quinn", , 172.0, 77.5, 57000, , ,
Advertencia: La línea 21 en data.txt tiene 1 campos en lugar de 6.
Contenido de la línea: 
Columnas en df1: ['"name"', 'age', '"height_(inches)"', '"weight_(pounds)"', '"salary_($)"', '"city"']
Columnas en df2: ['"name"', 'sex', '"marital_status"', 'career']
Columnas en df3: ['name', 'age', 'height_(cm)', 'weight_(kg)', 'salary_($)', 'city']

Primeras filas de df1:
      "name"  age "height_(inches)" "weight_(pounds)" "salary_($)"  \
0      "Amy"   30              65

## Paso 5: Unificar unidades

In [5]:
df1 = unificar_unidades(df1)
df2 = unificar_unidades(df2)
df3 = unificar_unidades(df3)

print("Estadísticas de df1 después de unificar unidades:")
print(df1.describe())
print("\nEstadísticas de df2 después de unificar unidades:")
print(df2.describe())
print("\nEstadísticas de df3 después de unificar unidades:")
print(df3.describe())

Estadísticas de df1 después de unificar unidades:
             age
count  10.000000
mean   29.400000
std     3.339993
min    25.000000
25%    27.250000
50%    28.500000
75%    31.500000
max    35.000000

Estadísticas de df2 después de unificar unidades:
         "name"   sex "marital_status" career
count        29    29               29     29
unique       24     3                3     21
top     "David"  Male           Single  Nurse
freq          2    15               16      3

Estadísticas de df3 después de unificar unidades:
             age
count  16.000000
mean   29.375000
std     3.442383
min    24.000000
25%    26.750000
50%    29.500000
75%    32.250000
max    35.000000


## Paso 6: Combinar los DataFrames y eliminar columnas duplicadas

In [6]:
df_combined = pd.concat([df1, df2, df3], ignore_index=True)
df_combined = eliminar_columnas_duplicadas(df_combined)
print("Forma del DataFrame combinado:", df_combined.shape)
print("\nColumnas en el DataFrame combinado:", df_combined.columns.tolist())
print("\nPrimeras filas del DataFrame combinado:")
print(df_combined.head())

Forma del DataFrame combinado: (59, 11)

Columnas en el DataFrame combinado: ['name', 'age', 'height_(inches)', 'weight_(pounds)', 'salary_($)', 'city', 'sex', 'marital_status', 'career', 'height_(cm)', 'weight_(kg)']

Primeras filas del DataFrame combinado:
        name   age height_(inches) weight_(pounds) salary_($)  \
0      "Amy"  30.0            65.5           132.3      50000   
1      "Ben"  25.0            68.0           165.3      60000   
2  "Charlie"  32.0            67.0           180.5      55000   
3    "David"  26.0            70.0           188.7      59000   
4     "Ella"  28.0            61.0           126.0      52000   

              city  sex marital_status career height_(cm) weight_(kg)  
0       "New York"  NaN            NaN    NaN         NaN         NaN  
1    "Los Angeles"  NaN            NaN    NaN         NaN         NaN  
2        "Chicago"  NaN            NaN    NaN         NaN         NaN  
3  "San Francisco"  NaN            NaN    NaN         NaN     

## Paso 7: Eliminar duplicados

In [7]:
df_combined = df_combined.drop_duplicates(subset='name', keep='first')
print("Forma del DataFrame después de eliminar duplicados:", df_combined.shape)

Forma del DataFrame después de eliminar duplicados: (25, 11)


## Paso 8: Eliminar outliers

In [8]:
df_combined = eliminar_outliers(df_combined)
print("Forma del DataFrame después de eliminar outliers:", df_combined.shape)
print("\nEstadísticas del DataFrame final:")
print(df_combined.describe())

Forma del DataFrame después de eliminar outliers: (11, 11)

Estadísticas del DataFrame final:
             age
count  11.000000
mean   29.454545
std     3.173756
min    25.000000
25%    27.500000
50%    29.000000
75%    31.000000
max    35.000000


## Paso 9: Guardar en Excel

In [9]:
output_path = os.path.join('output', 'datos_combinados_limpios.xlsx')
df_combined.to_excel(output_path, index=False)
print(f"Datos guardados en {output_path}")

Datos guardados en output\datos_combinados_limpios.xlsx
